In [ ]:
import cv2
import numpy as np
import os
# Thư mục chứa ảnh
input_folder = 'exercise'
output_folder = 'output_exercise'
os.makedirs(output_folder, exist_ok=True)

def image_inverse(img):
    return 255 - img

def gamma_correction(img, gamma=2.2):
    img_float = img / 255.0
    corrected = np.power(img_float, 1/gamma)
    corrected_img = np.uint8(corrected * 255)
    return corrected_img

def log_transformation(img):
    img_float = img.astype(np.float32) + 1
    c = 255 / np.log(1 + np.max(img_float))
    log_img = c * np.log(img_float)
    log_img = np.uint8(np.clip(log_img, 0, 255))
    return log_img

def histogram_equalization(img):
    if len(img.shape) == 2:
        return cv2.equalizeHist(img)
    else:
        ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
        ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
        return cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

def contrast_stretching(img):
    min_val = np.min(img)
    max_val = np.max(img)
    stretched = (img - min_val) * (255 / (max_val - min_val))
    stretched = np.uint8(np.clip(stretched, 0, 255))
    return stretched

def apply_and_save(transform_func, key_name):
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
    if not image_files:
        print("Không tìm thấy ảnh trong thư mục 'exercise'.")
        return

    for filename in image_files:
        path = os.path.join(input_folder, filename)
        img = cv2.imread(path)
        if img is None:
            print(f"Không thể đọc ảnh: {filename}")
            continue

        transformed = transform_func(img)
        output_path = os.path.join(output_folder, f"{key_name}_{filename}")
        cv2.imwrite(output_path, transformed)
        print(f"Đã lưu ảnh: {output_path}")

        cv2.imshow(f"Original - {filename}", img)
        cv2.imshow(f"Transformed - {key_name}", transformed)
   
        key = cv2.waitKey(1000) & 0xFF  # đợi 1000ms
        cv2.destroyAllWindows()

# Menu chính
print("Chọn phương pháp biến đổi ảnh:")
print("I - Image Inverse")
print("G - Gamma Correction")
print("L - Log Transformation")
print("H - Histogram Equalization")
print("C - Contrast Stretching")

key = input("Nhập lựa chọn của bạn (I/G/L/H/C): ").upper()

options = {
    'I': (image_inverse, 'inverse'),
    'G': (gamma_correction, 'gamma'),
    'L': (log_transformation, 'log'),
    'H': (histogram_equalization, 'histogram'),
    'C': (contrast_stretching, 'contrast'),
}

if key in options:
    func, name = options[key]
    apply_and_save(func, name)
else:
    print("Lựa chọn không hợp lệ.")


Chọn phương pháp biến đổi ảnh:
I - Image Inverse
G - Gamma Correction
L - Log Transformation
H - Histogram Equalization
C - Contrast Stretching
Đã lưu ảnh: output_exercise\contrast_ha-long-bay-in-vietnam.jpg
Đã lưu ảnh: output_exercise\contrast_pagoda.jpg
Đã lưu ảnh: output_exercise\contrast_quang_ninh.jpg


In [4]:
import cv2
import numpy as np
import os

# Thư mục chứa ảnh
input_folder = 'exercise'
output_folder = 'output_cau2'
os.makedirs(output_folder, exist_ok=True)

def fast_fourier_transform(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1)
    magnitude_spectrum = np.uint8(np.clip(magnitude_spectrum, 0, 255))
    return cv2.cvtColor(magnitude_spectrum, cv2.COLOR_GRAY2BGR)

def butterworth_filter(shape, cutoff, order, highpass=False):
    rows, cols = shape
    u = np.arange(rows) - rows // 2
    v = np.arange(cols) - cols // 2
    V, U = np.meshgrid(v, u)
    D = np.sqrt(U**2 + V**2)
    if highpass:
        H = 1 / (1 + (cutoff / (D + 1e-5)) ** (2 * order))  # +1e-5 tránh chia 0
    else:
        H = 1 / (1 + (D / cutoff) ** (2 * order))
    return H

def apply_butterworth(img, cutoff=30, order=2, highpass=False):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    H = butterworth_filter(gray.shape, cutoff, order, highpass)
    filtered = fshift * H
    img_back = np.fft.ifft2(np.fft.ifftshift(filtered))
    img_back = np.abs(img_back)
    img_back = np.uint8(np.clip(img_back, 0, 255))
    return cv2.cvtColor(img_back, cv2.COLOR_GRAY2BGR)

def apply_and_save(transform_func, key_name):
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
    if not image_files:
        print("Không tìm thấy ảnh trong thư mục 'exercise'.")
        return

    for filename in image_files:
        path = os.path.join(input_folder, filename)
        img = cv2.imread(path)
        if img is None:
            print(f"Không thể đọc ảnh: {filename}")
            continue

        transformed = transform_func(img)
        output_path = os.path.join(output_folder, f"{key_name}_{filename}")
        cv2.imwrite(output_path, transformed)
        print(f"Đã lưu ảnh: {output_path}")

        cv2.imshow(f"Original - {filename}", img)
        cv2.imshow(f"Transformed - {key_name}", transformed)

        key = cv2.waitKey(1000) & 0xFF  # đợi 1 giây
        cv2.destroyAllWindows()

# Menu chính
print("Chọn phương pháp biến đổi ảnh:")
print("F - Fast Fourier Transform")
print("L - Butterworth Lowpass Filter")
print("H - Butterworth Highpass Filter")

key = input("Nhập lựa chọn của bạn (F/L/H): ").upper()

options = {
    'F': (fast_fourier_transform, 'fft'),
    'L': (lambda img: apply_butterworth(img, cutoff=30, order=2, highpass=False), 'lowpass'),
    'H': (lambda img: apply_butterworth(img, cutoff=30, order=2, highpass=True), 'highpass'),
}

if key in options:
    func, name = options[key]
    apply_and_save(func, name)
else:
    print("Lựa chọn không hợp lệ.")


Chọn phương pháp biến đổi ảnh:
F - Fast Fourier Transform
L - Butterworth Lowpass Filter
H - Butterworth Highpass Filter
Đã lưu ảnh: output_cau2\highpass_ha-long-bay-in-vietnam.jpg
Đã lưu ảnh: output_cau2\highpass_pagoda.jpg
Đã lưu ảnh: output_cau2\highpass_quang_ninh.jpg


In [5]:
import cv2
import numpy as np
import os
import random

# Thư mục ảnh gốc và ảnh đầu ra
input_folder = 'exercise'
output_folder = 'output_cau3'
os.makedirs(output_folder, exist_ok=True)

# Các hàm biến đổi (từ câu 1)
def image_inverse(img): return 255 - img

def gamma_correction(img, gamma=2.2):
    img_float = img / 255.0
    corrected = np.power(img_float, 1/gamma)
    return np.uint8(corrected * 255)

def log_transformation(img):
    img_float = img.astype(np.float32) + 1
    c = 255 / np.log(1 + np.max(img_float))
    return np.uint8(np.clip(c * np.log(img_float), 0, 255))

def histogram_equalization(img):
    if len(img.shape) == 2:
        return cv2.equalizeHist(img)
    ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
    return cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

def contrast_stretching(img):
    min_val, max_val = np.min(img), np.max(img)
    stretched = (img - min_val) * (255 / (max_val - min_val))
    return np.uint8(np.clip(stretched, 0, 255))

# Danh sách hàm và tên
transformations = [
    (image_inverse, "inverse"),
    (gamma_correction, "gamma"),
    (log_transformation, "log"),
    (histogram_equalization, "histogram"),
    (contrast_stretching, "contrast")
]

# Đọc và xử lý ảnh
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for filename in image_files:
    img = cv2.imread(os.path.join(input_folder, filename))
    if img is None:
        print(f"Không đọc được ảnh: {filename}")
        continue

    # Đổi thứ tự RGB ngẫu nhiên
    channels = [0, 1, 2]
    random.shuffle(channels)
    img_rgb_shuffled = img[:, :, channels]

    # Chọn ngẫu nhiên 1 phép biến đổi trong câu 1
    func, name = random.choice(transformations)
    transformed = func(img_rgb_shuffled)

    # Lưu và hiển thị ảnh
    save_path = os.path.join(output_folder, f"{name}_rgb_{filename}")
    cv2.imwrite(save_path, transformed)
    print(f"Đã lưu ảnh: {save_path}")

    cv2.imshow("Original", img)
    cv2.imshow("Transformed", transformed)
    cv2.waitKey(1000)
    cv2.destroyAllWindows()


Đã lưu ảnh: output_cau3\log_rgb_ha-long-bay-in-vietnam.jpg
Đã lưu ảnh: output_cau3\contrast_rgb_pagoda.jpg
Đã lưu ảnh: output_cau3\contrast_rgb_quang_ninh.jpg


In [6]:
import cv2
import numpy as np
import os
import random

input_folder = 'exercise'
output_folder = 'output_cau4'
os.makedirs(output_folder, exist_ok=True)

def fast_fourier_transform(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    spectrum = 20 * np.log(np.abs(fshift) + 1)
    return cv2.cvtColor(np.uint8(np.clip(spectrum, 0, 255)), cv2.COLOR_GRAY2BGR)

def butterworth_filter(shape, cutoff, order, highpass=False):
    rows, cols = shape
    u = np.arange(rows) - rows // 2
    v = np.arange(cols) - cols // 2
    V, U = np.meshgrid(v, u)
    D = np.sqrt(U**2 + V**2)
    if highpass:
        H = 1 / (1 + (cutoff / (D + 1e-5)) ** (2 * order))
    else:
        H = 1 / (1 + (D / cutoff) ** (2 * order))
    return H

def apply_butterworth(img, cutoff, order, highpass):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    H = butterworth_filter(gray.shape, cutoff, order, highpass)
    f_filtered = fshift * H
    img_back = np.fft.ifft2(np.fft.ifftshift(f_filtered))
    return np.uint8(np.clip(np.abs(img_back), 0, 255))

def min_filter(img, ksize=3):
    return cv2.erode(img, np.ones((ksize, ksize), np.uint8))

def max_filter(img, ksize=3):
    return cv2.dilate(img, np.ones((ksize, ksize), np.uint8))

# Danh sách các biến đổi trong câu 2
def transform_fft(img):
    return fast_fourier_transform(img)

def transform_lowpass_min(img):
    bw = apply_butterworth(img, cutoff=30, order=2, highpass=False)
    filtered = min_filter(bw)
    return cv2.cvtColor(filtered, cv2.COLOR_GRAY2BGR)

def transform_highpass_max(img):
    bw = apply_butterworth(img, cutoff=30, order=2, highpass=True)
    filtered = max_filter(bw)
    return cv2.cvtColor(filtered, cv2.COLOR_GRAY2BGR)

transformations = [
    (transform_fft, "fft"),
    (transform_lowpass_min, "lowpass_min"),
    (transform_highpass_max, "highpass_max")
]

# Đọc và xử lý ảnh
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for filename in image_files:
    img = cv2.imread(os.path.join(input_folder, filename))
    if img is None:
        print(f"Không đọc được ảnh: {filename}")
        continue

    # Đổi thứ tự kênh màu RGB
    channels = [0, 1, 2]
    random.shuffle(channels)
    img_shuffled = img[:, :, channels]

    # Ngẫu nhiên chọn biến đổi trong câu 2
    func, name = random.choice(transformations)
    transformed = func(img_shuffled)

    save_path = os.path.join(output_folder, f"{name}_rgb_{filename}")
    cv2.imwrite(save_path, transformed)
    print(f"Đã lưu ảnh: {save_path}")

    cv2.imshow("Original", img)
    cv2.imshow("Transformed", transformed)
    cv2.waitKey(1000)
    cv2.destroyAllWindows()


Đã lưu ảnh: output_cau4\fft_rgb_ha-long-bay-in-vietnam.jpg
Đã lưu ảnh: output_cau4\fft_rgb_pagoda.jpg
Đã lưu ảnh: output_cau4\fft_rgb_quang_ninh.jpg
